<a href="https://colab.research.google.com/github/HenryZumaeta/py4cd_EPC2025/blob/main/C14/C14_Script01_PlotlyAvanzado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arquitectura de Plotly

**Plotly Express (px)**: Capa de Alto Nivel
- Paradigma de diseño: API declarativa y funcional, inspirada en la gramática de gráficos (Similar a ggplot/vega-lite).
- Abstracción principal: Unificación de datos y estéticas mediante pd.dataFrame.
- Patrón de uso: Una función = Un tipo de gráfico completo.
- Código base: Aproximadamente 2000 líneas de código Python que generan dinámicamente Objetos Graph Objetcs.

**Plotly Graph Objects (go)**: Capa de Bajo Nivel
- Paradigma de diseño: API orientada a objetos y de construcción explícita.
- Abstracción principal: Representación directa del grafo de escena JSON.
- Patrón de uso: Construcción manual y composición de trazas y layout.
- Código base: Definición exhaustiva de clases Python que mapean 1:1 con schema JSON.

# Diagramas de dispersión

In [ ]:
# Módulos y datos

# En Seaborn
import seaborn as sns
# sns.set_theme(style="darkgrid")
tips = sns.load_dataset("tips")

# Importemos plotly.express para crear gráficos interactivos de alto nivel
import plotly.express as px

# Importamos go
import plotly.graph_objects as go

# Se requiere para las regresiones el módulo sklearn
from sklearn.linear_model import LinearRegression

# Cargamos numpy
import numpy as np

# Importar el módulo scipy para el cálculo del KDE (Kernel Density Estimation)
from scipy.stats import gaussian_kde

In [ ]:
from _plotly_utils.colors.qualitative import Alphabet_r
from _plotly_utils.colors.carto import Bold_r
from plotly.colors import qualitative
# Veamos cómo sería en plotly

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importemos plotly.express para crear gráficos interactivos de alto nivel
import plotly.express as px

fig = px.scatter(data_frame = tips,
                 # Primera dimensión
                 x="tip",
                 # Segunda dimensión
                 y="total_bill",
                 # Tercera dimensión
                 color="day",
                 # Cuarta dimensión
                 size = "size",
                 # La 5ta dimensión: Forma de caracter para cada par ordenado
                 symbol= "sex",
                 # Podemos personalizar los símbolos/caracteres usados en la quinta dimensión
                 symbol_map = {
                     "Male": "x",
                     "Female": "circle-open-dot"
                 },
                 title = "Total Bill vs Tip (por día de la semana)",
                #  color_discrete_sequence = px.colors.qualitative.Alphabet_r
                #  color_discrete_sequence = px.colors.qualitative.Pastel
                 color_discrete_sequence = px.colors.qualitative.Bold_r

                )

# Mostrar el gráfico interactivo
fig.show()

In [ ]:
from re import template
from plotly.io import show
# Necesito diferenciar el total_bill máximo para cada día
# Paso 1: Encontrar el índice donde total_bill es el máximo de cada día
max_bill_idx = tips.loc[tips.groupby("day", observed=False)["total_bill"].idxmax()]

fig = px.scatter(data_frame = tips,
                 x="tip",
                 y="total_bill",
                 color="day",
                 size = "size",
                 # La 5ta dimensión: Forma de caracter para cada par ordenado
                 symbol= "sex",
                 # Podemos personalizar los símbolos/caracteres usados en la quinta dimensión
                 symbol_map = {
                     "Male": "x",
                     "Female": "circle-open-dot"
                 },
                 title = "Total Bill vs Tip(Por día): Resaltando los máximos de total_bill",
                 color_discrete_sequence = px.colors.qualitative.Bold_r

                )

# Anadimos una trazxa (capa) adicional usando graph objects para resaltal puntos específicos
fig.add_trace(
    # El gráfico agregar: Es otro diagrama de dispersión
    go.Scatter(
        # Valores de la columna "tip" de los máximos por día
        x = max_bill_idx["tip"],

        # Valores de la columna "total_bill" de los máximos por día
        y = max_bill_idx["total_bill"],

        # Mode de visualización: Solo marcadores (SIN LINEAS)
        mode = "markers",

        #  Personalización de los marcadores
        marker = {
            "color":"black",
            "size":20,
            "symbol":"circle-open",
            "line":{"width":3}
        },
        # No mostrar la leyenda de esta última traza
    showlegend = False,

    # Desactivamos el tooltip para  estos puntos específicos
    hoverinfo = "skip",
    # Evitamos la superposición de información

    # Nombre interno de la traza (Aunque no se muestra en leyenda)
    name = "Máximo por día"

    )
)

# Personalizar el layout (diseño) del gráfico
fig.update_layout(
    # Modificar el título de la leyenda
    legend_title_text = "Día y Género",

    # Modifiquemos la etiqueta del eje X
    xaxis_title = "Propina",

    # Modifiquemos la etiqueta del eje Y
    yaxis_title = "Total de la cuenta",

    # Plantilla de estilo básica de plotly
    template = "plotly_white"
)

# Mostrar el gráfico interactivo
fig.show()

In [ ]:
# Agregar lineas de regresion (lineal) para los datos de cada dia
fig = px.scatter(
    data_frame= tips,
    x = "tip",
    y = "total_bill",
    color = "day",
    size = "size",
    # La 5ta dimension  : Forma del caracter para cada par ordenado
    # symbol="sex",
    # POdemos personalizar los simbolos/caracteres usados en la 5ta dimension
    symbol_map={
        "Male" : "x",
        "Female" :"circle-open-dot"
    },
    title = "Total Bill vs Tip (por dia) : Resaltando los maximos de total_bill",
    # color_discrete_sequence=px.colors.qualitative.Alphabet_r
    color_discrete_sequence=px.colors.qualitative.Bold_r
)

# Diccionario para mapear nombres de dias a sus colores asignados en px.scatter
color_map = {}
# Itera sobre cada traza (conjunto de puntos) creado por px.scatter
for trace in fig.data:
  # Extraemos el nombre del dia de trace
  day_name = trace.name
  # Almacena el color asignado a este dia en el diccionario color_map
  color_map[day_name] = trace.marker.color
  # Iteramos sobre cada dia en el dataset: tips
for day in tips["day"].unique():
  # Filtramos el dataframe tips para obtener solo los datos de ese dia
  day_data = tips[tips["day"] == day].copy()
  # day_data : Almacena los valores de X e Y para realizar la regresion
  X = day_data[["tip"]].values
  y = day_data["total_bill"]
  # INstanciamos el modelo de regresion lineal y lo ajustamos
  model = LinearRegression()
  model.fit(X,y)
  # Generamos 150 puntos equiespaciados para la linea de regresion de cada dia
  x_line = np.linspace(X.min(), X.max(), 150).reshape(-1,1)
  # Con x_line predecimos los valores de la variable dependiente con el modelo
  y_line = model.predict(x_line)
  # Agregamos la nueva traza (linea de regresion) al grafico (diagrama de dispersion : px.scatter) existente
  fig.add_trace(
      go.Scatter(
          # puntos del dominio
          x = x_line.flatten(),
          # Puntos que predice el modelo
          y = y_line,
          # Configurar el color de la linea de regresion
          line = {
              "color" : color_map[day],
              "width" : 3
          },

          # nombre descriptivo en la leyenda
          name = f"Regresion {day}",
          # Agrupar las lineas de regresion con los puntos del dia
          legendgroup= day,
          # Desactivar los tooltip de las lineas de regresion
          hoverinfo="skip"
      )

  )
# Personalizacion del layout
fig.update_layout(
    # titulo personalizado de la leyenda
    legend_title_text = "Dia / Regresion",
    # Etiqueta a los ejes
    xaxis_title = "Propina (tip)",
    yaxis_title = "Cuenta Total (total_bill)",
    # Apliquemos la plantilla basica
    template = "plotly_white"

)
# Fin
fig.show()


# Histrogramas

In [ ]:
# Paso 1: Calcular los componentes del histograma y del kde (manualmente para mayo control)
# np.histogram: Calcula la distribución de frecuencias de mi variable de interés (tip)
hist_values, bin_edges = np.histogram(tips["tip"], bins=15, density = True)
# Las alturas de cada rectángulo del histograma y los extremos de cada intervalo de clase

# Calculamos los centros de cada intervalos de clases (bin_edge) para un posicionamiento preciso
bin_centers = (bin_edges[:-1] + bin_edges[1:])/2

# Calculemos el Kernel Density Estimation (kde)
kde = gaussian_kde(tips["tip"])

# Crear puntos equidistantes para evaluar la curva kde
x_kde = np.linspace(tips["tip"].min(), tips["tip"].max(), 150)

# Evaluar la curva kde en los puntos equidistantes (x_kde)
y_kde = kde(x_kde)

#---------------------------------------------------------------------------------------
# Pensemos en desarrollar únicamente sobre la capa de bajo nivel: Plotly y Graph Objects
#---------------------------------------------------------------------------------------

# Crear una figura vacía usando Graph Objects para luego agregar el histograma y
# el diagrama de dispersión, básicamente porque deseamos un control total sobre
# cada componente

fig = go.Figure()

# Agregamos el histograma (barras) como primera traza
fig.add_trace(
    # go agregue un histograma
    go.Histogram(
        # Data para el histograma
        x = tips["tip"],
        # Número de intervalos de clase
        nbinsx = 15,

        # Nombre que aparecerá en la leyenda
        name = "Histograma",

        # Normalizar: Para que pueda agregar la curva kde
        histnorm = "probability density",

        # Personalicemos las barras del histograma
        marker = dict(
            color = "lightseagreen",
            line = dict(
                color = "darkblue",
                width = 2.5)),
        # Modificar la transparencia
        opacity = 0.6,

        # Modificar la información que obtenemos al pasar el mouse sobre casa barra del histograma
        hovertemplate="Rango: %{x} <br>Densidad: %{y:.2f}"
    )
)

# Agregar un diagrama de línea: Curva kde
fig.add_trace(
    # Agreguemos la curva kde
    go.Scatter(
        # Variable independiente
        x = x_kde,

        # Variable dependiente
        y = y_kde,

        # Cambiamos/personalizamos el nombre de la línea (trace 1)
        name = "Curva KDE",

        # Personalicemos las propiedades de la línea KDE
        line = dict(
            color = "darkgreen",
            width = 1.9,
            dash = "dash"
        ),

        # Modifiquemos el tooltip
        hovertemplate = "Propina: %{x:.2f} <br>Densidad: %{y:.2f}<extra></extra>"
    )
) # Correctly closing fig.add_trace()

# Personalicemos el layout
fig.update_layout(
   # Modifiquemos la configuración del eje x
   xaxis = dict(
       # Propiedades de la etiqueta del eje x
       title = dict( # Corrected: 'text' should be inside 'title' dictionary
           text = "Monto de la propina",
           font = dict(
               size = 12,
               color = "darkblue",
               family = "Times New Roman"
           )
       ),

       # Modifiquemos propiedades de la fuente
       tickfont = dict(
           size = 12,
           color = "orange",
           family = "Montserrat"
       )
   ),
   # Modifiquemos la configuración del eje y
   yaxis = dict(
       # Propiedades de la etiqueta del eje x
       title = dict( # Corrected: 'text' should be inside 'title' dictionary
           text = "Densidad",
           font = dict(
               size = 12,
               color = "darkred",
               family = "Georgia, serif"
           )
       ),

       # Modifiquemos propiedades de la fuente
       tickfont = dict(
           size = 12,
           color = "darkgreen",
           family = "Arial, sans-serif"
       )
   ),

   # Personalicemos la configuración del título
   title = dict(
       # Texto del título
       text = "<b>Ditribución de propinas <br> con Curva de densidad </b>",

       # Posición horizontal centrada
       x = 0.5,

       # Posición vertical centrada
       y = 0.95,

       # Propiedades del título
       font = dict(
           size = 20,
           color = "darkblue",
           family = "Montserrat"
       )

   )

)

# Fin
fig.show()
